# Détection en temps réel

## Imports

In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
import sklearn
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from os.path import isfile, join
import csv
import mediapipe as mp 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix
from scipy import stats
mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_holistic = mp.solutions.holistic  # Mediapipe Solutions

## Préliminaires

In [2]:
df = pd.read_csv('/home/portable014/Bureau/neelp_1/MAIN_4_PC/Projet/main4_capgemini/pre-traitement/données/stockage_csv_non_normalized/64_adresse_vouloir.csv')
nb_coords = 33 * 4 + 2 * (21*4)
nb_frames = int((len(df.columns) - 1) / nb_coords)
print(nb_frames)

df =df.fillna(0)

Y = df[['class']]
X = df.iloc[:, 1:6001]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=27, stratify=Y)

20


In [3]:
test = len(df.index) / 20
test

60.0

In [4]:
num_coords = 75 # nombre de points du corps + main gauche + main droite
# for landmark in results.right_hand_landmarks.landmark:
#     print(landmark, landmark.value)
names = []
for i in range (1,nb_frames+1): 
    for val in range(1, num_coords+1):
        names += ['x' + str (val) + '_' + str(i), 'y' + str (val) + '_' + str(i),
                    'z' + str (val) + '_' + str(i), 'v' + str (val) + '_' + str(i)]

In [5]:
print(len(names))

6000


In [6]:
print(nb_frames, nb_coords)
print(nb_frames * nb_coords)

20 300
6000


## Entrainement classifier

In [7]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)



/home/portable014/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearDiscriminantAnalysis()

## Fonctions préliminaires

In [8]:
def mediapipe_detection(image, model): #Fonction pour que mediapipe puisse détecter (car il ne travaille pas dans le même système de couleur que OpenCV)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
                            
def extract_keypoints(results):
    pose = (np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4))
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = (np.array([[res.x, res.y, res.z, res.visibility] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*4))
    rh = (np.array([[res.x, res.y, res.z, res.visibility] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*4))
    return np.concatenate([pose, lh, rh])

colors = [(245,117,16), (117,245,16), (16,117,245)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

## Boucle de détection en temps réel

In [9]:
# 1. New detection variables
actions = np.array(['adresse','affaire','aller','ami_amie','autre','beaucoup','bonjour','chocolat','comprendre','demander','dieu','donc','dormir','enceinte','faire','famille','finir','gens','heure','ils_elles','interdire','jamais','jour','laisser','lentement','marcher_marche','merci','mourir_mort','nous_on','nuit','ou','payer','penser','personne','peu','pleuvoir_pluie','portugal','pour','prendre','question','quoi','raison_connaissance','rencontrer_rencontre','rester','rien','rue_route','sac','soeur_nonne','soleil','surprendre_surprise','temps','toujours','travail','trop','trouver','turquie','vache','vivre_vie','voir','vouloir'])
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 20)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-nb_frames:]
        res = ' '
        if len(sequence) == nb_frames:
            prediction = np.array(sequence).flatten().reshape(1, -1)
            predictions_df = pd.DataFrame(data = prediction, columns = names)
            res = model.predict(predictions_df)[0]
        #     print(res)
        #     print(actions[np.argmax(res)])
        #     predictions.append(np.argmax(res))
            
            
        # #3. Viz logic
        #     if np.unique(predictions[-10:])[0]==np.argmax(res): 
        #         if res[np.argmax(res)] > threshold: 
                    
        #             if len(sentence) > 0: 
        #                 if actions[np.argmax(res)] != sentence[-1]:
        #                     sentence.append(actions[np.argmax(res)])
        #             else:
        #                 sentence.append(actions[np.argmax(res)])

        #     if len(sentence) > 5: 
        #         sentence = sentence[-5:]

        #     # Viz probabilities
        #     image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(res), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

: 